In [25]:
from pynq import Overlay, MMIO
import pynq.lib as lib
import time
import numpy as np
from datetime import datetime
from pytz import timezone
overlay = Overlay("./KV260_SPI_wrapper.bit")
AxiQspi = overlay.axi_quad_spi_0
overlay?

In [26]:
# Offset
# Interrupt Control Grouping
XSP_DGIER_OFFSET = 0x1C # Device global interrupt enable register
XSP_IISR_OFFSET = 0x20 # IP interrupt status register
XSP_IIER_OFFSET = 0x28 # IP interrupt enable register
# Core Grouping
XSP_SRR_OFFSET = 0x40 # Software reset register
XSP_CR_OFFSET = 0x60 # SPI control register
XSP_SR_OFFSET = 0x64 # SPI status register
XSP_DTR_OFFSET = 0x68 # SPI data transmit register.
XSP_DRR_OFFSET = 0x6C # SPI data receive register
XSP_SSR_OFFSET = 0x70 # SPI Slave select register
XSP_TFO_OFFSET = 0x74 # Transmit FIFO occupancy register
XSP_RFO_OFFSET = 0x78 # Receive FIFO occupancy register
XSP_REGISTERS = [0x40, 0x60, 0x64, 0x68, 0x6c, 0x70, 0x74, 0x78, 0x1c, 0x20, 0x28]

# Mask
XSP_SRR_RESET_MASK = 0x0A
XSP_SR_TX_EMPTY_MASK = 0x04
XSP_SR_TX_FULL_MASK = 0x08
## Crl mask
XSP_CR_TRANS_INHIBIT_MASK = 0x100 # Master Transaction Inhibit 0 = enable, 1 = disable
XSP_CR_LOOPBACK_MASK = 0x01 # Lookback mode: 0 = Normal operation, 1 = Loopback mode
XSP_CR_ENABLE_MASK = 0x02 # SPI system enable: 0 = SPI system disabled, 1 = SPI system enabled.
XSP_CR_MASTER_MODE_MASK = 0x04 # Set SPI mode: 0 = Slave configuration, 1 = Master configuration.
XSP_CR_CLK_POLARITY_MASK = 0x08 # Clock polarity
XSP_CR_CLK_PHASE_MASK = 0x10 # Clock phase
XSP_CR_TXFIFO_RESET_MASK = 0x20 # Transmit FIFO reset: 0 = Transmit FIFO normal operation, 1 = Reset transmit FIFO pointer.
XSP_CR_RXFIFO_RESET_MASK = 0x40 # Receive FIFO reset: 0 = Receive FIFO normal operation, 1 = Reset receive FIFO pointer.
XSP_CR_MANUAL_SS_MASK = 0x80 # Manual slave select assertion enable

SLAVE_NO_SELECTION = 0xFFFFFFFF

In [27]:
def cnfg(AxiQspi, clk_phase=0, clk_pol=0):
    print("Configure device")
    # Reset the SPI device
    AxiQspi.write(XSP_SRR_OFFSET, XSP_SRR_RESET_MASK)
    # Enable the transmit empty interrupt, which we use to determine progress on the transmission. 
    AxiQspi.write(XSP_IIER_OFFSET, XSP_SR_TX_EMPTY_MASK)
    # Disable the global IPIF interrupt
    AxiQspi.write(XSP_DGIER_OFFSET, 0)
    # Deselect the slave on the SPI bus
    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
    # Disable the transmitter, enable Manual Slave Select Assertion, put SPI controller into master mode, and enable it
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg | XSP_CR_MASTER_MODE_MASK | XSP_CR_MANUAL_SS_MASK | XSP_CR_ENABLE_MASK | XSP_CR_TXFIFO_RESET_MASK | XSP_CR_RXFIFO_RESET_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)
    
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg &= ~(XSP_CR_CLK_PHASE_MASK | XSP_CR_CLK_POLARITY_MASK)
    if clk_pol == 1:    # CPOL
        ControlReg |= XSP_CR_CLK_POLARITY_MASK
    if clk_phase == 1:  # CPHA
        ControlReg |= XSP_CR_CLK_PHASE_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg) #?
    CRRead = AxiQspi.read(XSP_CR_OFFSET)
    print(f"XSP_CR_OFFSET : {bin(CRRead)[2:].zfill(10)}")
    return 0
print("Done")

Done


In [28]:
def xfer(B2Send, B2Recv, AxiQspi):
    """
    SPI Transfer function that sends and receives data in one 32-bit manner.
    Args:
        AxiQspi: SPI device object.
        B2Send : List of bytes to send .
        B2Recv : List to store received bytes .
    """

    # ----------------------
    # 1. 組合 4 Byte -> 32 bit，並一次寫入
    # ----------------------
    if len(B2Send) != 4 or B2Send.dtype != np.uint8:
        raise ValueError("B2Send must be a NumPy array of 4 uint8 bytes.")

    data32 = np.frombuffer(B2Send.tobytes(), dtype=np.uint32).byteswap()[0]

#     print(f"byte2Send: {B2Send} => 0x{data32:08X} (32-bit)")
    AxiQspi.write(XSP_DTR_OFFSET, int(data32))
#     StatusReg = AxiQspi.read(XSP_SR_OFFSET)
#     print(f"STAT_REG (二進制) : {bin(StatusReg)[2:].zfill(11)}")
    # 選擇 SPI Slave
    AxiQspi.write(XSP_SSR_OFFSET, 0xFFFFFFFE)

    # 清除 TRANS_INHIBIT_MASK
    ControlReg = AxiQspi.read(XSP_CR_OFFSET) & ~XSP_CR_TRANS_INHIBIT_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    # 等待傳送完成 (TX FIFO empty)
    while (AxiQspi.read(XSP_SR_OFFSET) & XSP_SR_TX_EMPTY_MASK) == 0:
        pass

    # 讀取回一筆 (假設硬體會回傳 32 bit 到 RFO_OFFSET)
    temp_rfo = AxiQspi.read(XSP_RFO_OFFSET)
#     print(f"XSP_RFO_OFFSET: 0x{temp_rfo:08X}")

    # 傳送完後，重新設置 TRANS_INHIBIT_MASK
#     ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg |= XSP_CR_TRANS_INHIBIT_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    # 取消所有 Slave
    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
#     StatusReg = AxiQspi.read(XSP_SR_OFFSET)
#     print(f"STAT_REG (二進制) : {bin(StatusReg)[2:].zfill(11)}")
#     # ----------------------
#     # 2. 讀取 32 bit 並拆解成 4 個 8 bit
#     # ----------------------
#     print("ReadResponse")
#     RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01
    
    # 清空 B2Recv (確保不會累加舊的資料)
    B2Recv = np.empty(0, dtype=np.uint8)
    while (AxiQspi.read(XSP_SR_OFFSET) & 0x01) == 0:
        temp32_drr = AxiQspi.read(XSP_DRR_OFFSET)
        # 使用 NumPy 快速轉換 32-bit 為 4-byte
        bytes_arr = np.frombuffer(temp32_drr.to_bytes(4, byteorder='big'), dtype=np.uint8)
        # 使用 np.concatenate 高效合併結果
        B2Recv = np.concatenate([B2Recv, bytes_arr])
        binary_representation = " ".join(f"{byte:08b}" for byte in B2Recv)
        print(f"B2Recv (Binary): {binary_representation}")

print("set xfer Done")

set xfer Done


In [29]:
def signedToUnsigned(n, byte_count):
    min_val = -(1 << (8 * byte_count - 1))
    max_val = (1 << (8 * byte_count - 1)) - 1
    if not (min_val <= n <= max_val):
        raise ValueError(f"Value {n} exceeds {byte_count} byte signed range!")
    return int.from_bytes(n.to_bytes(byte_count, 'little', signed=True), 'little', signed=False)


def toSigned32(n):
    return np.int32(n).item()


def getDateTime(): 
    format = "%Y-%m-%d %H:%M:%S"
    now_utc = datetime.now(timezone('UTC'))
    now_asia = now_utc.astimezone(timezone('Asia/Taipei'))
    print(now_asia.strftime(format))
    
# Read和Write(reg_addr) are DEX?y
# 要改，T和R的width都不一樣
import numpy as np
from datetime import datetime
from pytz import timezone

def signedToUnsigned(n, byte_count):
    min_val = -(1 << (8 * byte_count - 1))
    max_val = (1 << (8 * byte_count - 1)) - 1
    if not (min_val <= n <= max_val):
        raise ValueError(f"Value {n} exceeds {byte_count} byte signed range!")
    return int.from_bytes(n.to_bytes(byte_count, 'little', signed=True), 'little', signed=False)

def toSigned32(n):
    return np.int32(n).item()

def getDateTime():
    format = "%Y-%m-%d %H:%M:%S"
    now_utc = datetime.now(timezone('UTC'))
    now_asia = now_utc.astimezone(timezone('Asia/Taipei'))
    print(now_asia.strftime(format))

def ReadIntan(reg_addr, times):
    reg_addr = int(reg_addr)  
    byte2Send = np.array([0xf0, reg_addr, 0x00, 0x00], dtype=np.uint8)
    byte2Recv = np.empty(4, dtype=np.uint8) 
    for _ in range(times):
        xfer(byte2Send, byte2Recv, AxiQspi)
    MB = signedToUnsigned(byte2Recv[2], 1)
    LB = signedToUnsigned(byte2Recv[3], 1)
    realdata = MB << 8 | LB
    return toSigned32(realdata)

# D要以2byte的方式傳送(hex)
def WriteIntan(R, D, timew):
    wbyte2Send = np.array([0xB0, R, D, 0x00], dtype=np.uint8)
    wbyte2Recv = np.empty(4, dtype=np.uint8)
    for _ in range(timew):
        xfer(wbyte2Send, wbyte2Recv, AxiQspi)
    

def CaliIntan():
    cbyte2Send = np.array([0x55, 0x00, 0x00, 0x00], dtype=np.uint8)
    cbyte2Recv = np.empty(4, dtype=np.uint8)
    xfer(cbyte2Send, cbyte2Recv, AxiQspi)
    
CFactor = 0.001  # 設定 CFactor

def ConvertIntan(C):
    cobyte2Send = np.array([0x3C, C, 0x00, 0x00], dtype=np.uint8)
    cobyte2Recv = np.empty(4, dtype=np.uint8)
    xfer(cobyte2Send, cobyte2Recv, AxiQspi)
    MB = signedToUnsigned(cobyte2Recv[0], 1)
    MB2 = signedToUnsigned(cobyte2Recv[1], 1)
    LB = signedToUnsigned(cobyte2Recv[2], 1)
    LB2 = signedToUnsigned(cobyte2Recv[3], 1)
    ACrealdata = MB << 24 | MB2 << 16
    return toSigned32(ACrealdata) * CFactor

def Iclear():
    clbyte2Send = np.array([0x6A, 0x00, 0x00, 0x00], dtype=np.uint8)
    clbyte2Recv = np.empty(4, dtype=np.uint8)
    xfer(clbyte2Send, clbyte2Recv, AxiQspi)

print("Done")

Done


In [43]:
cnfg(AxiQspi, clk_phase=0, clk_pol=0)
byte2Send = np.array([0xf0, 0xfc, 0x00, 0x00], dtype=np.uint8)
byte2Recv = np.empty(4, dtype=np.uint8)
xfer(byte2Send, byte2Recv, AxiQspi)

cnfg(AxiQspi, clk_phase=1, clk_pol=0)
iend = range(1, 9000)
for _ in iend:
    byte2Send = np.array([0xf0, 0xfc, 0x00, 0x00], dtype=np.uint8)
    byte2Recv = np.empty(4, dtype=np.uint8)
    xfer(byte2Send, byte2Recv, AxiQspi)

Configure device
XSP_CR_OFFSET : 0110000110
B2Recv (Binary): 00000000 00000000 00000000 00000000
Configure device
XSP_CR_OFFSET : 0110010110
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Binary): 00000000 00000000 01010100 01000001
B2Recv (Bin

B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00000000
B2Recv (Binary): 00000000 00000000 00000000 00

In [10]:
byte2Recv

array([240, 251,   0,   0], dtype=uint8)

In [77]:
# clear()
iend = list(range(1,3000))
for i in iend:
    rom_addr=251
    readreg=ReadIntan(rom_addr,3)
    print("Read Intan Done")
    print("RG:",readreg)
#     print(ReadIntan(rom_addr,3))
# DMB = signedToUnsigned(readreg[0], 1) #list?
# DLB = signedToUnsigned(readreg[1], 1)
# print(chr(DMB),end='')
# print(chr(DLB),end='')

TransferData
byte2Send: [240, 251, 0, 0] => 0xF0FB0000 (32-bit)
STAT_REG (二進制) : 00000100001
XSP_RFO_OFFSET: 0x00000000
STAT_REG (二進制) : 00000100100
ReadResponse
XSP_RFO_OFFSET: 0x00000000
32-bit read from DRR : 0x000024A7
B2Recv after split : ['0x00', '0x00', '0x24', '0xA7']
Done
TransferData
byte2Send: [240, 251, 0, 0] => 0xF0FB0000 (32-bit)
STAT_REG (二進制) : 00000100001
XSP_RFO_OFFSET: 0x00000000
STAT_REG (二進制) : 00000100100
ReadResponse
XSP_RFO_OFFSET: 0x00000000
32-bit read from DRR : 0x000024A7
B2Recv after split : ['0x00', '0x00', '0x24', '0xA7']
Done
TransferData
byte2Send: [240, 251, 0, 0] => 0xF0FB0000 (32-bit)
STAT_REG (二進制) : 00000100001
XSP_RFO_OFFSET: 0x00000000
STAT_REG (二進制) : 00000100100
ReadResponse
XSP_RFO_OFFSET: 0x00000000
32-bit read from DRR : 0x000024A7
B2Recv after split : ['0x00', '0x00', '0x24', '0xA7']
Done


IndexError: list index out of range